In [8]:
import json, os
import datetime
import pandas as pd
from pandas.io.json import json_normalize

In [9]:
# Helper function
def parse_json(infile):
    with open(infile) as f:
        data = json.load(f)
    df = json_normalize(json_normalize(data).loc[0,'messages'])
    df['date_time'] = [datetime.datetime.fromtimestamp(t/1000.0) for t in df['timestamp_ms']]
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['year'] = [t.year for t in df['date_time']]
    df['month'] = [t.month for t in df['date_time']]
    df['day'] = [t.day for t in df['date_time']]
    df['hour'] = [t.hour for t in df['date_time']]
    return df[['content', 'timestamp_ms', 'type', 'date_time', 'year', 'month', 'day', 'hour']]

In [10]:
dataset_p = '../../data'
messages_p = dataset_p + '/messages/inbox/'

In [11]:
all_json = [messages_p + s + '/message.json' for s in sorted(os.listdir(messages_p))[1:]]
print("Total number of chats:", len(all_json))

Total number of chats: 339


In [15]:
for j in range(len(all_json)):
    if j == 0:
        try:
            df = parse_json(all_json[j])
        except:
            continue
    else:
        try:
            df = pd.concat([df, parse_json(all_json[j])], 0)
        except:
            continue
df.shape

(22108, 8)

In [18]:
df.head()

,content,timestamp_ms,type,date_time,year,month,day,hour
0,how are you dear :),1459907494895,Generic,2016-04-05 18:51:34.895,2016,4,5,18
1,take a rest,1459881248322,Generic,2016-04-05 11:34:08.322,2016,4,5,11
2,so you tired now,1459881243643,Generic,2016-04-05 11:34:03.643,2016,4,5,11
3,No it's okay,1459881221299,Generic,2016-04-05 11:33:41.299,2016,4,5,11
4,im sorry,1459881194549,Generic,2016-04-05 11:33:14.549,2016,4,5,11


In [17]:
df.to_csv(dataset_p + '/clean_messages.csv', index = None)